In [62]:
! uv pip install langchain openai tiktoken rapidocr-onnxruntime python-dotenv langchain-community pymupdf faiss-cpu langchain-google-genai

Using Python 3.12.3 environment at: /workspaces/MANASILYA/.venv
⠙ Resolving dependencies...                                                     

Resolved 83 packages in 3.99s                                        
⠙ Preparing packages... (0/8)                                                   
⠙ Preparing packages... (0/8)-------------------     0 B/19.50 KiB           
⠙ Preparing packages... (0/8)---------- 14.83 KiB/19.50 KiB         
⠙ Preparing packages... (0/8)---------- 14.83 KiB/19.50 KiB         
filetype             ------------------------------ 14.83 KiB/19.50 KiB
⠙ Preparing packages... (0/8)-------------------     0 B/33.88 KiB           
filetype             ------------------------------ 14.83 KiB/19.50 KiB
⠙ Preparing packages... (0/8)-------------------     0 B/33.88 KiB           
filetype             ------------------------------ 14.83 KiB/19.50 KiB
⠙ Preparing packages... (0/8)-------------------     0 B/33.88 KiB           
filetype             ------------------------------ 14.83 KiB/19.50 KiB
rsa                  ------------------------------     0 B/33.88 KiB
⠙ Preparing packages... (0/8)------------

In [65]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [66]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
api_key = os.getenv("OPENAI_API_KEY")

## DATA INGESTION


In [70]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [48]:
loader = PyMuPDFLoader("/workspaces/MANASILYA/Data/100x_syllabus.pdf")

In [49]:
documents = loader.load()

In [50]:
print("~"*30)
print(f"total Document Length = {len(documents)}")
print("~"*30)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
total Document Length = 15
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [51]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
)

In [52]:
text_chunks = text_splitter.split_documents(documents)

In [53]:
text_chunks[0]

Document(metadata={'producer': 'XeTeX 0.99992', 'creator': 'LaTeX with hyperref package', 'creationdate': '2025-12-08T17:57:04+00:00', 'source': '/workspaces/MANASILYA/Data/100x_syllabus.pdf', 'file_path': '/workspaces/MANASILYA/Data/100x_syllabus.pdf', 'total_pages': 15, 'format': 'PDF 1.5', 'title': 'Complete CS Roadmap 2025', 'author': 'Tech Learning Guide', 'subject': 'Computer Science Learning Roadmap', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20251208175704-00'00'", 'page': 0}, page_content='Complete CS\nRoadmap\n2025 Edition\nFull Stack\nDevOps\nAI/ML\nWeb3\nYour comprehensive guide to\nbecoming a software engineer\nFrom Beginner to Industry Ready\nBased on industry best practices and real-world experience\nLast Updated: December 2025')

In [58]:
#embeddings = OpenAIEmbeddings(api_key=api_key)
#embeddings.embed_documents("hello")


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************53YA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}

In [71]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [72]:
vector_store = FAISS.from_documents(text_chunks,embedding=embeddings)

In [73]:
vector_store

In [84]:
retriever = vector_store.as_retriever()

In [76]:
#Performing similarity search
query = "what is pre-requites to learn ai/ml"
docs = vector_store.similarity_search(query,k=4)

#display the results
for i , doc in enumerate(docs):
    print(f"document{i+1}:")
    print(doc.page_content)
    print("-"*30)
          

document1:
4
AI/ML ENGINEERING
CS Roadmap 2025
AI/ML Engineering
AI/ML Engineering
Information
This roadmap covers both research-oriented topics (understanding transformers) and
applied AI (building agents, RAG systems). Focus based on your goals - most jobs are
in applied AI.
Learning Path
Syllabus
Part A: Foundations and History
1. History of AI
2-3 days
How we arrived at transformers
Pre-transformer approaches
2. Deep Learning Basics
1 week
Backpropagation, sigmoid functions
Classical ML concepts
3. Neural Networks + PyTorch
2 weeks
Building neural nets from scratch
PyTorch fundamentals
Tensors, autograd
4. RNNs, LSTMs, Sequential Models
1 week (Optional)
5. CNNs
1 week (Optional)
Part B: Transformers and Attention
6. Coding Simple Attention
1-2 weeks
“Attention is All You Need” paper
Implementing attention from scratch
7. Attention Optimizations
2 weeks
KV Cache, MQA, GQA
MLA (Multi-head Latent Attention)
DSA (DeepSeek’s architecture)
8. Hugging Face Ecosystem
1 week
Part C: Applie

In [115]:
#Prompting
from langchain_core.prompts import ChatPromptTemplate

template = """
You are assistant for question-answering tasks.
Use the folllowing pieces of retrieved context to 
answer the question.If you don't know the answer,
just say that you don't know.Use ten sentences maximum 
and keep the answer concise.
Question : {question}
Context : {context}
Answer :
"""

prompt = ChatPromptTemplate.from_template(template)

In [116]:
#Str Output Parser
from langchain_classic.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()

In [117]:
#LLM 
#!uv pip install langchain_openai 
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini")

In [118]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x79491b29b110>, search_kwargs={})

In [123]:
#RAG CHAIN
from langchain_classic.schema.runnable import RunnablePassthrough

rag_chain = (
    {
        "context": RunnablePassthrough() | (lambda x: x["question"]) | retriever,
        "question": RunnablePassthrough() | (lambda x: x["question"])
    }
    | prompt
    | llm
    | output_parser
)

In [125]:
query = "what is pre-requistes of ai/ml"

rag_chain.invoke({"question": query})  

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************53YA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'code': 'invalid_api_key', 'param': None}, 'status': 401}